In [1]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [2]:
creds = '{"username":"amaboh","key":"91c62f966a299e953c262f1d4cd9a599"}'

In [4]:
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exists_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [5]:
path = Path('santander-product-recommendation')

In [5]:
if not iskaggle and not path.exists():
    import zipfile,kaggle
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 228M/228M [03:09<00:00, 1.26MB/s]


In [6]:
import pandas as pd
import numpy as np
import random 
from timeit import default_timer as timer

In [7]:
dtype_list = {'ind_cco_fin_ult1': 'uint8', 'ind_deme_fin_ult1': 'uint8',
          'ind_aval_fin_ult1': 'uint8', 'ind_valo_fin_ult1': 'uint8',
          'ind_reca_fin_ult1': 'uint8', 'ind_ctju_fin_ult1': 'uint8',
          'ind_cder_fin_ult1': 'uint8', 'ind_plan_fin_ult1': 'uint8',
          'ind_fond_fin_ult1': 'uint8', 'ind_hip_fin_ult1': 'uint8',
          'ind_pres_fin_ult1': 'uint8', 'ind_nomina_ult1': 'Int64', 
          'ind_cno_fin_ult1': 'uint8', 'ind_ctpp_fin_ult1': 'uint8',
          'ind_ahor_fin_ult1': 'uint8', 'ind_dela_fin_ult1': 'uint8',
          'ind_ecue_fin_ult1': 'uint8', 'ind_nom_pens_ult1': 'Int64',
          'ind_recibo_ult1': 'uint8', 'ind_deco_fin_ult1': 'uint8',
          'ind_tjcr_fin_ult1': 'uint8', 'ind_ctop_fin_ult1': 'uint8',
          'ind_viv_fin_ult1': 'uint8', 'ind_ctma_fin_ult1': 'uint8',
         'ncodpers' : 'uint32'}  

name_col = ['ncodpers', 'fecha_dato', 'ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1',
           'ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
           'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1',
           'ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1',
           'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1',
           'ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
           'ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1',
           'ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1']

In [8]:
start = timer()

#optimizing memory usage using chunksize 
reader = pd.read_csv('santander-product-recommendation/train_ver2.csv.zip', chunksize=1e6, 
                     dtype=dtype_list, usecols=name_col)

#concatenate the data from all the chunks into a single dataframe
df_train = pd.concat([chunk for chunk in reader])

In [27]:
#exploring data
df_train.shape

(13647309, 26)

In [28]:
df_train.dtypes

fecha_dato           object
ncodpers             uint32
ind_ahor_fin_ult1     uint8
ind_aval_fin_ult1     uint8
ind_cco_fin_ult1      uint8
ind_cder_fin_ult1     uint8
ind_cno_fin_ult1      uint8
ind_ctju_fin_ult1     uint8
ind_ctma_fin_ult1     uint8
ind_ctop_fin_ult1     uint8
ind_ctpp_fin_ult1     uint8
ind_deco_fin_ult1     uint8
ind_deme_fin_ult1     uint8
ind_dela_fin_ult1     uint8
ind_ecue_fin_ult1     uint8
ind_fond_fin_ult1     uint8
ind_hip_fin_ult1      uint8
ind_plan_fin_ult1     uint8
ind_pres_fin_ult1     uint8
ind_reca_fin_ult1     uint8
ind_tjcr_fin_ult1     uint8
ind_valo_fin_ult1     uint8
ind_viv_fin_ult1      uint8
ind_nomina_ult1       Int64
ind_nom_pens_ult1     Int64
ind_recibo_ult1       uint8
dtype: object

In [29]:
df_train.head(5)

,fecha_dato,ncodpers,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2015-01-28,1050611,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2015-01-28,1050612,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2015-01-28,1050613,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2015-01-28,1050614,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df_train1505 = df_train[df_train.fecha_dato == '2015-05-28']
df_train1505 = df_train1505.drop(['fecha_dato'], axis=1, inplace=False)
df_train1505 = df_train1505.fillna(0)

#### Popularity Recommendation System

In [13]:
def popularity_based(df):
    '''This function would calcualte the popularity of a product occuring within the range of 0 to 0.99'''
    
    top_col = {}
    
    for col in df.columns[1:]:
        top_col[col] = df[col].value_counts()[1]
    
    for k,v in top_col.items():
        top_col[k] = np.around( v / df.shape[0], decimals=4)
        
    return top_col

In [14]:
popularity_based(df_train1505)


    

{'ind_ahor_fin_ult1': 0.0001,
 'ind_aval_fin_ult1': 0.0,
 'ind_cco_fin_ult1': 0.775,
 'ind_cder_fin_ult1': 0.0005,
 'ind_cno_fin_ult1': 0.1003,
 'ind_ctju_fin_ult1': 0.0121,
 'ind_ctma_fin_ult1': 0.0112,
 'ind_ctop_fin_ult1': 0.1661,
 'ind_ctpp_fin_ult1': 0.0562,
 'ind_deco_fin_ult1': 0.0033,
 'ind_deme_fin_ult1': 0.0023,
 'ind_dela_fin_ult1': 0.0556,
 'ind_ecue_fin_ult1': 0.1006,
 'ind_fond_fin_ult1': 0.0239,
 'ind_hip_fin_ult1': 0.0076,
 'ind_plan_fin_ult1': 0.0117,
 'ind_pres_fin_ult1': 0.0037,
 'ind_reca_fin_ult1': 0.0632,
 'ind_tjcr_fin_ult1': 0.0568,
 'ind_valo_fin_ult1': 0.0316,
 'ind_viv_fin_ult1': 0.005,
 'ind_nomina_ult1': 0.0655,
 'ind_nom_pens_ult1': 0.0672,
 'ind_recibo_ult1': 0.1527}

#### Memory Based - Collaborative Filtering

In [15]:
df_ui = df_train1505.copy()

df_ui = df_ui[:10000] #limit to 10k instances/users
df_ui = df_ui.set_index('ncodpers')
df_ui.shape #check 10000 limit

(10000, 24)

In [18]:
from sklearn.metrics.pairwise import pairwise_distances

# calculate the similarity between user-item
cosine_sim = 1 - pairwise_distances(df_ui, metric='cosine')

In [24]:
def useritem(user_id, df, sim_matrix= cosine_sim):
    """
    This function is takes in 3 argurments, user id, dataframe or user-item
    and cosine similarity, to calculate the most similar users and return the probability for a giver user 
    based on similar users
    """
    cos_id = list(df.index).index(user_id)
    
    k = 0
    sim_min = 0.79
    user_sim_k = {}
    
    while k < 20:
        # loop over users in the df to create a dictionary of similar users
        for user in range(len(df)):
            #check if the present user and user in df have a similarity score between sim_min and 0.99, 0.99 is threshold because if not it would be score between the user and himself
            if sim_min < cosine_sim[cos_id, user] < 0.99:
                user_sim_k[user] = cosine_sim[cos_id,user]
                k+=1
                
        sim_min -= 0.25 #i
            
        if sim_min < 0.65:
            break
    #sorted K most similar users
    user_sim_k = dict(sorted(user_sim_k.items(), key=lambda item:item[1], reverse=True))
    user_id_k = list(user_sim_k.keys())
    
    #Dataframe with most similar users
    df_user_k = df.iloc[user_id_k]
    df_user_k_T = df_user_k.T
    
    #change the user index to the cosine index
    df_user_k_T.columns = user_id_k
    
    #mean of ownership by k similar users
    
    ownership = []
    usit = {}
    
    for row_name, row in df_user_k_T.iterrows():
        for index, own in row.items():
            
            ownership.append(own)
        usit[row_name] = np.mean(ownership)
        ownership = []
        
    # if there are no users with similarity at least 0.65, the recommendation probability is 0
    
    if pd.isna(list(usit.values())[0]) == True:
        
        usit = {key : 0 for (key, value) in usit.items()}
    return usit
        
    
    

In [26]:
useritem(1061604, df_ui)

{'ind_ahor_fin_ult1': 0.0,
 'ind_aval_fin_ult1': 0.0,
 'ind_cco_fin_ult1': 1.0,
 'ind_cder_fin_ult1': 0.0,
 'ind_cno_fin_ult1': 0.0,
 'ind_ctju_fin_ult1': 0.0,
 'ind_ctma_fin_ult1': 0.0,
 'ind_ctop_fin_ult1': 0.0,
 'ind_ctpp_fin_ult1': 0.0,
 'ind_deco_fin_ult1': 0.0,
 'ind_deme_fin_ult1': 0.0,
 'ind_dela_fin_ult1': 0.009433962264150943,
 'ind_ecue_fin_ult1': 0.8490566037735849,
 'ind_fond_fin_ult1': 0.0,
 'ind_hip_fin_ult1': 0.0,
 'ind_plan_fin_ult1': 0.009433962264150943,
 'ind_pres_fin_ult1': 0.0,
 'ind_reca_fin_ult1': 0.22641509433962265,
 'ind_tjcr_fin_ult1': 0.018867924528301886,
 'ind_valo_fin_ult1': 0.0,
 'ind_viv_fin_ult1': 0.0,
 'ind_nomina_ult1': 0.0,
 'ind_nom_pens_ult1': 0.0,
 'ind_recibo_ult1': 0.9622641509433962}

### Model Based Collaborative filtering 

This model based collaborative filtering is base on machine learning models 

In [30]:
df_mb = df_train1505.copy()
df_mb = df_mb.set_index('ncodpers')

In [31]:
df_val1603 = df_train[df_train.fecha_dato == '2016-03-28']
df_val1603 = df_val1603.drop(['fecha_dato'], axis=1, inplace=False)
df_val1603 = df_val1603.set_index('ncodpers')

In [32]:
from collections import defaultdict
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [33]:
import warnings
warnings.filterwarnings('ignore')

In [37]:
def modelbased(user_id, df, model=DecisionTreeClassifier(max_depth=9)):
    """
    Uses machine learning model to calculate recommendation for a given user
    """
    
    mdbs = {}
    
    for c in df.columns:
        y_train = df[c].astype('int')
        x_train = df.drop([c], axis=1)
        model.fit(x_train, y_train)
        p_train = model.predict_proba(x_train[x_train.index == user_id])[:,1]
    
        mdbs[c] = p_train[0]
        
    return mdbs
    

In [38]:
modelbased(1061604,df_mb) 

{'ind_ahor_fin_ult1': 0.0005380683346785041,
 'ind_aval_fin_ult1': 0.0003398470688190314,
 'ind_cco_fin_ult1': 0.9395129615082483,
 'ind_cder_fin_ult1': 0.0018475750577367205,
 'ind_cno_fin_ult1': 0.11796875,
 'ind_ctju_fin_ult1': 0.0,
 'ind_ctma_fin_ult1': 0.00906344410876133,
 'ind_ctop_fin_ult1': 0.12646370023419204,
 'ind_ctpp_fin_ult1': 0.14297227595099934,
 'ind_deco_fin_ult1': 0.0007849978194505015,
 'ind_deme_fin_ult1': 0.002002302648045252,
 'ind_dela_fin_ult1': 0.1542642924086223,
 'ind_ecue_fin_ult1': 0.24242424242424243,
 'ind_fond_fin_ult1': 0.0496031746031746,
 'ind_hip_fin_ult1': 0.015165031222123104,
 'ind_plan_fin_ult1': 0.018646408839779006,
 'ind_pres_fin_ult1': 0.002824858757062147,
 'ind_reca_fin_ult1': 0.17329545454545456,
 'ind_tjcr_fin_ult1': 0.2798129384255651,
 'ind_valo_fin_ult1': 0.11129800953181945,
 'ind_viv_fin_ult1': 0.006155567991046447,
 'ind_nomina_ult1': 0.0,
 'ind_nom_pens_ult1': 0.0016565433462175593,
 'ind_recibo_ult1': 0.5610047846889952}

### Weighted Hybrid Recommendation System

In [43]:
def hybrid(user_id, df_p, df_u, df_m, f1, f2, f3):
    """
    Function that calculates the weighted hybrid recommendations for a 
    given user.
    """
    
    pb_h = popularity_based(df_p)
    ui_h = useritem(user_id, df_u)
    mb_h = modelbased(user_id, df_m)
    
    hybrid = {}
    
    for k, v in pb_h.items():
        hybrid[k] = (v * f1) + (ui_h[k] * f2) + (mb_h[k] * f3)
    
    return hybrid

In [44]:
hybrid_rec_1061604 = hybrid(1061604, df_p = df_train1505, df_u = df_ui, df_m = df_mb, f1= 0.5, f2 = 0.25, f3 = 0.25)

hybrid_rec_1061604

{'ind_ahor_fin_ult1': 0.00018451708366962603,
 'ind_aval_fin_ult1': 8.496176720475786e-05,
 'ind_cco_fin_ult1': 0.872378240377062,
 'ind_cder_fin_ult1': 0.0007118937644341801,
 'ind_cno_fin_ult1': 0.0796421875,
 'ind_ctju_fin_ult1': 0.00605,
 'ind_ctma_fin_ult1': 0.007865861027190332,
 'ind_ctop_fin_ult1': 0.114665925058548,
 'ind_ctpp_fin_ult1': 0.06384306898774983,
 'ind_deco_fin_ult1': 0.0018462494548626253,
 'ind_deme_fin_ult1': 0.001650575662011313,
 'ind_dela_fin_ult1': 0.06872456366819331,
 'ind_ecue_fin_ult1': 0.32317021154945685,
 'ind_fond_fin_ult1': 0.02435079365079365,
 'ind_hip_fin_ult1': 0.007591257805530776,
 'ind_plan_fin_ult1': 0.012870092775982488,
 'ind_pres_fin_ult1': 0.002556214689265537,
 'ind_reca_fin_ult1': 0.1315276372212693,
 'ind_tjcr_fin_ult1': 0.10307021573846675,
 'ind_valo_fin_ult1': 0.043624502382954865,
 'ind_viv_fin_ult1': 0.004038891997761612,
 'ind_nomina_ult1': 0.03275,
 'ind_nom_pens_ult1': 0.03401413583655439,
 'ind_recibo_ult1': 0.457167233908097

###### Obtain Recommendation after computation with names of products

In [45]:
product_names = {"ind_ahor_fin_ult1" : "Saving Account",
"ind_aval_fin_ult1" : "Guarantees",
"ind_cco_fin_ult1" : "Current Accounts",
"ind_cder_fin_ult1" : "Derivada Account",
"ind_cno_fin_ult1" : "Payroll Account",
"ind_ctju_fin_ult1" : "Junior Account",
"ind_ctma_fin_ult1" : "Más Particular Account",
"ind_ctop_fin_ult1" : "Particular Account",
"ind_ctpp_fin_ult1" : "Particular Plus Account",
"ind_deco_fin_ult1" : "Short-term Deposits",
"ind_deme_fin_ult1" : "Medium-term Deposits",
"ind_dela_fin_ult1" : "Long-term Deposits",
"ind_ecue_fin_ult1" : "E-account",
"ind_fond_fin_ult1" : "Funds",
"ind_hip_fin_ult1" : "Mortgage",
"ind_plan_fin_ult1" : "Pensions",
"ind_pres_fin_ult1" : "Loans",
"ind_reca_fin_ult1" : "Taxes",
"ind_tjcr_fin_ult1" : "Credit Card",
"ind_valo_fin_ult1" : "Securities",
"ind_viv_fin_ult1" : "Home Account",
"ind_nomina_ult1" : "Payroll",
"ind_nom_pens_ult1" : "Pensions",
"ind_recibo_ult1" : "Direct Debit"}

In [46]:
# this functions maps overs product names and returns a new list with with product names

def change_names(col_names, map_products= product_names):
    """
    change index name to product names
    """
    
    return list(map(lambda col_name: map_products[col_name], col_names))

In [48]:
def recommendation(user_id, df, hybrid_outcome):
    """
    Function that returns a list 
    """
    
    # products that the user already owns
    user_row = df[df.index == user_id]
    user_products = list(filter(lambda product: user_row[product].to_numpy() == 1, user_row))
    
    # removes products that the user already owns
    recom = {key: hybrid_outcome[key] for key in hybrid_outcome if key not in user_products}
    
    recom_sorted = dict(sorted(recom.items(), key=lambda item: item[1], reverse=True))
    
    return list(recom_sorted.keys())

In [49]:
rec_1061604 = recommendation(1061604, df_mb, hybrid_rec_1061604)